In [1]:
import boto.ec2
import boto3

In [2]:
%run aws_helper.py

In [3]:
si = SpotInstances()
ctq = ClusterTaskQueue()

In [4]:
sqs = boto3.resource('sqs')  

In [5]:
new_member_queue = sqs.get_queue_by_name(QueueName='cluster_new_member')
task_commitment_queue = sqs.get_queue_by_name(QueueName='cluster_task_commitment')
task_queue = sqs.get_queue_by_name(QueueName='cluster_task_queue')
# instance metadata: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-instance-metadata.html
# ipython setup: http://ipython.org/ipython-doc/dev/notebook/public_server.html#running-a-public-notebook-server

In [6]:
si.request(num_instances=100, subnet='b')

The following script will be run on startup
#!/bin/bash

echo "==========================="
echo "Starting service 'worker-server'."
initctl status worker-server
echo "==========================="



In [8]:
reqs = si.conn.get_all_spot_instance_requests()

In [13]:
len([req.instance_id for req in reqs[3:]])

101

In [15]:
reservations = si.conn.get_all_reservations(instance_ids = [req.instance_id for req in reqs[4:]])

In [16]:
for res in reservations:
    for instance in res.instances:
        print(instance.public_dns_name)

ec2-52-89-193-15.us-west-2.compute.amazonaws.com
ec2-54-69-136-78.us-west-2.compute.amazonaws.com
ec2-54-68-193-159.us-west-2.compute.amazonaws.com
ec2-52-88-201-154.us-west-2.compute.amazonaws.com
ec2-52-89-15-13.us-west-2.compute.amazonaws.com
ec2-52-89-45-252.us-west-2.compute.amazonaws.com
ec2-54-69-129-216.us-west-2.compute.amazonaws.com
ec2-52-89-235-49.us-west-2.compute.amazonaws.com
ec2-54-68-201-203.us-west-2.compute.amazonaws.com
ec2-52-10-43-242.us-west-2.compute.amazonaws.com
ec2-52-88-239-14.us-west-2.compute.amazonaws.com
ec2-54-69-13-4.us-west-2.compute.amazonaws.com
ec2-52-89-133-116.us-west-2.compute.amazonaws.com
ec2-54-68-203-141.us-west-2.compute.amazonaws.com
ec2-54-69-128-40.us-west-2.compute.amazonaws.com
ec2-52-89-243-103.us-west-2.compute.amazonaws.com
ec2-54-68-167-111.us-west-2.compute.amazonaws.com
ec2-54-68-149-49.us-west-2.compute.amazonaws.com
ec2-52-88-140-31.us-west-2.compute.amazonaws.com
ec2-54-69-13-17.us-west-2.compute.amazonaws.com
ec2-54-68-108-25

In [7]:
si.print_public_dns_names()

ERROR:boto:400 Bad Request
ERROR:boto:<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>InvalidInstanceID.Malformed</Code><Message>Invalid id: "None"</Message></Error></Errors><RequestID>21f92871-234e-4e6c-80ba-b87276e86de3</RequestID></Response>


EC2ResponseError: EC2ResponseError: 400 Bad Request
<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>InvalidInstanceID.Malformed</Code><Message>Invalid id: "None"</Message></Error></Errors><RequestID>21f92871-234e-4e6c-80ba-b87276e86de3</RequestID></Response>

In [ ]:
ctq.add_task('hello yejllow 17:27')

In [ ]:
ctq.add_task('git:pull')

In [ ]:
#ctq.add_task('parse:01')
ctq.add_task('parse:test/006:6457:40100')

In [ ]:
ctq.add_task('XXXparse:tmp/01:404075:405000')

In [23]:
run_id = 'prod/06'
total_ids = 100
for i in range(total_ids):
    #ctq.add_task('quit')
    #ctq.add_task('git:pull')
    ctq.add_task('parse:%s:%d:%d' % (run_id, i, total_ids))
    #ctq.add_task('parse:%s' % (run_id))

In [ ]:
ctq.add_task('quit')

In [18]:
started = []

In [21]:
for i in range(100):
    msgs = new_member_queue.receive_messages(MessageAttributeNames=['Worker'])
    for msg in msgs:
        worker_dns_name = 'unknown'
        if msg.message_attributes is not None:
            worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
        print('%s: %s' % (worker_dns_name, msg.body))
        msg.delete()
        started.append(worker_dns_name)

ec2-52-88-57-125.us-west-2.compute.amazonaws.com: new_member
ec2-54-68-156-176.us-west-2.compute.amazonaws.com: new_member
ec2-52-89-45-252.us-west-2.compute.amazonaws.com: new_member
ec2-52-25-5-100.us-west-2.compute.amazonaws.com: new_member
ec2-52-24-99-129.us-west-2.compute.amazonaws.com: new_member
ec2-54-69-22-62.us-west-2.compute.amazonaws.com: new_member
ec2-54-68-203-253.us-west-2.compute.amazonaws.com: new_member
ec2-52-88-140-31.us-west-2.compute.amazonaws.com: new_member
ec2-52-89-133-116.us-west-2.compute.amazonaws.com: new_member
ec2-54-68-167-111.us-west-2.compute.amazonaws.com: new_member
ec2-54-69-7-70.us-west-2.compute.amazonaws.com: new_member
ec2-54-68-46-25.us-west-2.compute.amazonaws.com: new_member
ec2-52-25-242-153.us-west-2.compute.amazonaws.com: new_member
ec2-52-88-201-154.us-west-2.compute.amazonaws.com: new_member
ec2-52-27-101-119.us-west-2.compute.amazonaws.com: new_member
ec2-54-68-202-216.us-west-2.compute.amazonaws.com: new_member
ec2-54-69-100-163.us-

In [22]:
len(started)

100

In [ ]:
'ec2-52-27-130-165.us-west-2.compute.amazonaws.com' in started

In [ ]:
set(hosts) - set(started)

In [24]:
res = []

In [27]:
for i in range(100):
    msgs = task_commitment_queue.receive_messages(MessageAttributeNames=['Worker'])
    for msg in msgs:
        worker_dns_name = 'unknown'
        if msg.message_attributes is not None:
            worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
        print('%s: %s' % (worker_dns_name, msg.body))
        res.append((worker_dns_name, msg.body))
        msg.delete()

ec2-54-68-201-203.us-west-2.compute.amazonaws.com: parse:prod/06:26:100
ec2-54-69-136-58.us-west-2.compute.amazonaws.com: parse:prod/06:7:100
ec2-52-88-57-125.us-west-2.compute.amazonaws.com: parse:prod/06:14:100
ec2-54-69-22-62.us-west-2.compute.amazonaws.com: parse:prod/06:10:100
ec2-54-68-149-49.us-west-2.compute.amazonaws.com: parse:prod/06:23:100
ec2-52-89-151-225.us-west-2.compute.amazonaws.com: parse:prod/06:15:100
ec2-52-89-241-245.us-west-2.compute.amazonaws.com: parse:prod/06:17:100
ec2-54-68-167-111.us-west-2.compute.amazonaws.com: parse:prod/06:30:100
ec2-54-69-102-121.us-west-2.compute.amazonaws.com: parse:prod/06:19:100
ec2-54-69-13-17.us-west-2.compute.amazonaws.com: parse:prod/06:3:100
ec2-52-88-167-15.us-west-2.compute.amazonaws.com: parse:prod/06:31:100
ec2-52-10-114-136.us-west-2.compute.amazonaws.com: parse:prod/06:28:100
ec2-54-68-197-197.us-west-2.compute.amazonaws.com: parse:prod/06:0:100
ec2-54-68-115-107.us-west-2.compute.amazonaws.com: parse:prod/06:34:100
ec2

In [31]:
workers = [r[0] for r in res]

In [34]:
set(hosts) - set(workers)

set()

In [ ]:
#Delete 'quit' message after the end
msgs = task_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    #msg.delete()

In [ ]:
new_member_queue.purge()
task_queue.purge()
task_commitment_queue.purge()

In [17]:
hosts = [
    "ec2-52-89-193-15.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-78.us-west-2.compute.amazonaws.com",
    "ec2-54-68-193-159.us-west-2.compute.amazonaws.com",
    "ec2-52-88-201-154.us-west-2.compute.amazonaws.com",
    "ec2-52-89-15-13.us-west-2.compute.amazonaws.com",
    "ec2-52-89-45-252.us-west-2.compute.amazonaws.com",
    "ec2-54-69-129-216.us-west-2.compute.amazonaws.com",
    "ec2-52-89-235-49.us-west-2.compute.amazonaws.com",
    "ec2-54-68-201-203.us-west-2.compute.amazonaws.com",
    "ec2-52-10-43-242.us-west-2.compute.amazonaws.com",
    "ec2-52-88-239-14.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-4.us-west-2.compute.amazonaws.com",
    "ec2-52-89-133-116.us-west-2.compute.amazonaws.com",
    "ec2-54-68-203-141.us-west-2.compute.amazonaws.com",
    "ec2-54-69-128-40.us-west-2.compute.amazonaws.com",
    "ec2-52-89-243-103.us-west-2.compute.amazonaws.com",
    "ec2-54-68-167-111.us-west-2.compute.amazonaws.com",
    "ec2-54-68-149-49.us-west-2.compute.amazonaws.com",
    "ec2-52-88-140-31.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-17.us-west-2.compute.amazonaws.com",
    "ec2-54-68-108-253.us-west-2.compute.amazonaws.com",
    "ec2-52-89-135-95.us-west-2.compute.amazonaws.com",
    "ec2-52-89-221-244.us-west-2.compute.amazonaws.com",
    "ec2-52-89-254-95.us-west-2.compute.amazonaws.com",
    "ec2-54-68-166-47.us-west-2.compute.amazonaws.com",
    "ec2-54-69-43-170.us-west-2.compute.amazonaws.com",
    "ec2-52-89-151-225.us-west-2.compute.amazonaws.com",
    "ec2-54-69-133-194.us-west-2.compute.amazonaws.com",
    "ec2-52-26-206-14.us-west-2.compute.amazonaws.com",
    "ec2-52-88-57-125.us-west-2.compute.amazonaws.com",
    "ec2-54-69-102-121.us-west-2.compute.amazonaws.com",
    "ec2-52-88-5-19.us-west-2.compute.amazonaws.com",
    "ec2-52-25-5-100.us-west-2.compute.amazonaws.com",
    "ec2-54-68-125-99.us-west-2.compute.amazonaws.com",
    "ec2-52-89-55-24.us-west-2.compute.amazonaws.com",
    "ec2-54-68-179-101.us-west-2.compute.amazonaws.com",
    "ec2-54-69-7-70.us-west-2.compute.amazonaws.com",
    "ec2-54-68-46-25.us-west-2.compute.amazonaws.com",
    "ec2-52-27-101-119.us-west-2.compute.amazonaws.com",
    "ec2-54-69-16-198.us-west-2.compute.amazonaws.com",
    "ec2-52-27-202-38.us-west-2.compute.amazonaws.com",
    "ec2-52-89-241-245.us-west-2.compute.amazonaws.com",
    "ec2-52-89-188-228.us-west-2.compute.amazonaws.com",
    "ec2-54-68-163-25.us-west-2.compute.amazonaws.com",
    "ec2-54-69-100-163.us-west-2.compute.amazonaws.com",
    "ec2-52-89-215-11.us-west-2.compute.amazonaws.com",
    "ec2-52-89-93-129.us-west-2.compute.amazonaws.com",
    "ec2-54-68-148-244.us-west-2.compute.amazonaws.com",
    "ec2-54-68-205-216.us-west-2.compute.amazonaws.com",
    "ec2-54-69-111-80.us-west-2.compute.amazonaws.com",
    "ec2-54-69-16-189.us-west-2.compute.amazonaws.com",
    "ec2-54-69-13-59.us-west-2.compute.amazonaws.com",
    "ec2-52-89-248-201.us-west-2.compute.amazonaws.com",
    "ec2-52-88-167-15.us-west-2.compute.amazonaws.com",
    "ec2-54-68-102-238.us-west-2.compute.amazonaws.com",
    "ec2-54-68-197-197.us-west-2.compute.amazonaws.com",
    "ec2-52-24-252-202.us-west-2.compute.amazonaws.com",
    "ec2-52-26-56-255.us-west-2.compute.amazonaws.com",
    "ec2-52-11-188-93.us-west-2.compute.amazonaws.com",
    "ec2-52-89-37-225.us-west-2.compute.amazonaws.com",
    "ec2-54-69-134-170.us-west-2.compute.amazonaws.com",
    "ec2-54-68-230-132.us-west-2.compute.amazonaws.com",
    "ec2-52-26-247-58.us-west-2.compute.amazonaws.com",
    "ec2-54-69-22-62.us-west-2.compute.amazonaws.com",
    "ec2-52-88-148-10.us-west-2.compute.amazonaws.com",
    "ec2-54-68-104-52.us-west-2.compute.amazonaws.com",
    "ec2-54-68-115-107.us-west-2.compute.amazonaws.com",
    "ec2-52-89-241-191.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-58.us-west-2.compute.amazonaws.com",
    "ec2-54-69-11-201.us-west-2.compute.amazonaws.com",
    "ec2-52-10-114-136.us-west-2.compute.amazonaws.com",
    "ec2-54-68-202-216.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-34.us-west-2.compute.amazonaws.com",
    "ec2-52-88-34-128.us-west-2.compute.amazonaws.com",
    "ec2-52-88-176-236.us-west-2.compute.amazonaws.com",
    "ec2-52-89-208-176.us-west-2.compute.amazonaws.com",
    "ec2-52-24-102-152.us-west-2.compute.amazonaws.com",
    "ec2-52-89-227-161.us-west-2.compute.amazonaws.com",
    "ec2-54-68-203-253.us-west-2.compute.amazonaws.com",
    "ec2-52-89-90-171.us-west-2.compute.amazonaws.com",
    "ec2-52-89-196-70.us-west-2.compute.amazonaws.com",
    "ec2-52-88-12-15.us-west-2.compute.amazonaws.com",
    "ec2-52-89-175-90.us-west-2.compute.amazonaws.com",
    "ec2-52-89-230-26.us-west-2.compute.amazonaws.com",
    "ec2-54-68-156-176.us-west-2.compute.amazonaws.com",
    "ec2-52-89-163-39.us-west-2.compute.amazonaws.com",
    "ec2-54-69-112-242.us-west-2.compute.amazonaws.com",
    "ec2-52-25-123-5.us-west-2.compute.amazonaws.com",
    "ec2-52-88-216-208.us-west-2.compute.amazonaws.com",
    "ec2-52-25-242-153.us-west-2.compute.amazonaws.com",
    "ec2-54-68-131-38.us-west-2.compute.amazonaws.com",
    "ec2-52-88-135-209.us-west-2.compute.amazonaws.com",
    "ec2-52-27-6-21.us-west-2.compute.amazonaws.com",
    "ec2-54-69-136-66.us-west-2.compute.amazonaws.com",
    "ec2-52-89-222-49.us-west-2.compute.amazonaws.com",
    "ec2-52-89-226-215.us-west-2.compute.amazonaws.com",
    "ec2-52-89-187-92.us-west-2.compute.amazonaws.com",
    "ec2-52-24-99-129.us-west-2.compute.amazonaws.com",
    "ec2-52-89-132-66.us-west-2.compute.amazonaws.com",
    "ec2-52-89-73-242.us-west-2.compute.amazonaws.com"   
    ]
